# Synoptic report section extraction
Old code from Arjun: https://github.mskcc.org/knowledgesystems/cdm-pathology/blob/master/main_server.py



In [ ]:
import os
import sys
sys.path.insert(0, '/mind_data/fongc2/radiology_met_prediction/')
sys.path.insert(0, '../')
import re
import pandas as pd
from dremio_connection import RunDremio
from utils_pathology import set_debug_console
from key_value_pair_extraction import PathologyKeyValuePairExtraction
%config Completer.use_jedi = False

In [ ]:
set_debug_console()

In [ ]:
sqlquery = f'''
SELECT * FROM "phi_lake.cdm-data.pathology"."table_pathology_surgical_samples_parsed_specimen.csv"
'''
sql_accessions = f'''
SELECT * FROM "phi_lake"."cdm-data".pathology."path_synoptic_predictions.csv"
'''

sql_impact = f'''
SELECT * FROM "phi_lake"."cdm-data".pathology."table_pathology_impact_sample_summary_dop_anno.csv"
'''

sql_path_dict = f'''
SELECT * FROM "phi_lake"."cdm-data".pathology."path_synoptic_dict_all.csv"
'''

sql_path_stage = f'''
SELECT * FROM "phi_lake"."cdm-data".pathology."path_dx_stage_key_values.csv"
'''

sql_path_full = f'''
SELECT * FROM "phi_lake"."cdm-data".pathology."table_pathology_clean.csv"
'''


user = 'rosalind'
pw = 'franklinphoto51'

In [ ]:
fname_path_dx_synoptic = '/mind_data/minio_data/cdm-data/pathology/table_pathology_surgical_samples_parsed_specimen_synoptic.csv'
fname_path_dx_key_value = '/mind_data/minio_data/cdm-data/pathology/path_synoptic_dict_all.csv'
fname_path_dx_key_value_stage = '/mind_data/minio_data/cdm-data/pathology/path_dx_stage_key_values.csv'

In [ ]:
obj_dremio = RunDremio(user=user, pw=pw)
df = obj_dremio.query_data(sql=sqlquery)
df_access = obj_dremio.query_data(sql=sql_accessions)
df_impact = obj_dremio.query_data(sql=sql_impact)

df_path_dict = obj_dremio.query_data(sql=sql_path_dict)
df_pathstage = obj_dremio.query_data(sql=sql_path_stage)


In [ ]:
df_path_full = obj_dremio.query_data(sql=sql_path_full)

In [ ]:
(df_access['IS_PREDICTION'] == 'False').sum()

In [ ]:
df_path_dict.loc[df_path_dict['ACCESSION_NUMBER'] == 'S21-9532', 'PATH_DX_SPEC_DESC_DICT'].iloc[0]

In [ ]:
df_pathstage.head()

In [ ]:
print(df_pathstage.shape)
(df_pathstage['RESULT_STAGE'] == '').sum()

In [ ]:
df_pathstage['COL_NAME_STAGE'].value_counts().head(10).reset_index()
# df_pathstage['COL_NAME_STAGE'].nunique()


In [ ]:
vcounts = df_pathstage['RESULT_STAGE'].value_counts().reset_index()
vcounts.head(11)
# text = vcounts['index'].str[:45]
# vcounts = vcounts.assign(text=text)
# vcounts[['text', 'RESULT_STAGE']].head(12)
# df_pathstage['RESULT_STAGE'].nunique()

In [ ]:
stage_results = df_pathstage['RESULT_STAGE'].str.findall(r"[yp]*[TNM]{1}[0-4xX]{1}[a-dA-D]*|[I]{1,3}[a-dA-D]+|[0-4]{1}[a-dA-D]+")
stage_res_len = stage_results.apply(lambda x: len(x))
(stage_res_len > 0).sum()
df_pathstage = df_pathstage.assign(RESULT_STAGE_CLEAN=stage_results)
df_pathstage = df_pathstage.assign(RESULT_STAGE_CLEAN_LEN=stage_res_len)
print(df_pathstage.shape)
df_pathstage[df_pathstage['RESULT_STAGE_CLEAN_LEN'] > 0].sample(10)

In [ ]:
df_pathstage.loc[12363]['RESULT_STAGE']

In [ ]:
df_pathstage_filt = df_pathstage.loc[(df_pathstage['RESULT_STAGE_CLEAN_LEN'] == 0) & ~((df_pathstage['RESULT_STAGE'].isin(['', 'N/A', 'na'])) | (df_pathstage['RESULT_STAGE'].str.lower().str.contains('see')))]
print(df_pathstage_filt.shape)
df_pathstage_filt.sample(25)

In [ ]:
df_pathstage_filt2 = df_pathstage.loc[(df_pathstage['RESULT_STAGE_CLEAN_LEN'] == 0)  ]
print(df_pathstage_filt2.shape)
df_pathstage_filt2.sample(25)

In [ ]:
df_pathstage_filt2_reports = df_pathstage_filt2[['ACCESSION_NUMBER']].drop_duplicates().merge(right=df_path_full, how='left', on='ACCESSION_NUMBER')
df_pathstage_filt2_reports.to_csv('path_reports_staging_missing.tsv', index=False, sep='\t')

In [ ]:
access_num = df_access.loc[df_access['IS_SYNOPTIC'] == '1'].drop_duplicates()
access_num_non_syn = df_access.loc[df_access['IS_SYNOPTIC'] == '0'].drop_duplicates()
access_num

In [ ]:
df_impact_syn = df_impact.merge(right=access_num, how='left', left_on=['SOURCE_ACCESSION_NUMBER_0', 'SOURCE_SPEC_NUM_0'], right_on=['ACCESSION_NUMBER', 'PATH_DX_SPEC_NUM'])
(df_impact_syn['IS_SYNOPTIC'] == '1').sum()/df_impact_syn.shape[0]

In [ ]:
df_use = access_num.merge(right=df, how='left', on=['ACCESSION_NUMBER', 'PATH_DX_SPEC_NUM'])
df_not_use = access_num_non_syn.merge(right=df, how='left', on=['ACCESSION_NUMBER', 'PATH_DX_SPEC_NUM'])
df_use.head(10)

In [ ]:
print(df_use.shape)
print(df_use.nunique())
# df_use.to_csv(fname_path_dx_synoptic, index=False)

In [ ]:
# obj_key_value = PathologyKeyValuePairExtraction(fname=fname_path_dx_synoptic, col_dx_spec_desc='PATH_DX_SPEC_DESC', col_out_dict='PATH_DX_SPEC_DESC_DICT', list_col_index=['ACCESSION_NUMBER', 'PATH_DX_SPEC_NUM'], fname_out=fname_path_dx_key_value)
# obj_key_value.return_df()

In [ ]:
df_stage = obj_key_value.extract_term(list_key_terms=['STAG', 'AJCC'], key_label='STAGE', fname_save=fname_path_dx_key_value_stage)

In [ ]:
df_stage

In [ ]:
df1 = obj_key_value.return_df()
df1['PATH_DX_SPEC_DESC_DICT'][10]


In [ ]:
# print(df_use['PATH_DX_SPEC_DESC'].iloc[1]);

In [ ]:
# def get_key(item):
#     keys = re.findall(r"([\\r\\n]{0,3}[- ]*[ \w\( \)\-]+:)", item)
# #     keys_clean = [x.strip() for x in keys]
#     keys_clean = keys
#     return keys_clean

In [ ]:
# def cleanup_for_regex(text):
#     for ch in ['\\', ';', '`', '*', '_', '{', '}', '?', ':', '[', ']', '(', ')', '<', '>', '#', '+', '-', '.', '!', '$',
#                '"', '\'']:
#         if ch in text:
#             text = text.replace(ch, "\\" + ch)
#     return text

In [ ]:
# def get_value(current_item_updated, next_item_updated, item):
#     ''' {key_current:} capture text {key_[current+1]} '''

#     reg_exp_to_capture = r"(?<={0})(.*?)(?={1})".format(current_item_updated, next_item_updated)

#     value_to_capture = re.findall(reg_exp_to_capture, item, re.S | re.M)

#     rm_empty_string = list(filter(None, [i for i in value_to_capture[0].splitlines()]))
#     filtered_value_to_capture = ','.join(i for i in rm_empty_string)
#     filtered_again_value_to_capture = re.findall("[^,\s].*", filtered_value_to_capture)

#     return value_to_capture, filtered_again_value_to_capture

In [ ]:
# def set_key_in_item(current_item_updated, item):
#     '''To avoid duplicate keys in a report: strp keys as you iterate through provided in get_value() '''

#     reg_exp_to_del = r"({0})".format(current_item_updated)
#     item = re.sub(reg_exp_to_del, '', item, 1, re.S)

#     return item

In [ ]:
# def create_key_value_pairs(dx):
#     # Get keys 
#     keys = get_key(item=dx)
    
#     keys_final = []
#     values = []
#     for ind, i in enumerate(range(len(keys))):
#             try:
#                 if i != len(keys) - 1:
#                     current_item, next_item = keys[i], keys[i + 1]
#                 else:
#                     current_item, next_item = keys[i], '"""'
#             except Exception:
#                 print('Could not find end key')
#                 sys.exit()
#             current_item_updated = cleanup_for_regex(current_item)
#             next_item_updated = cleanup_for_regex(next_item)

#             # Get key
#             key = re.findall("([A-Za-z0-9].*[^:])", keys[i])
#             if len(key) == 0:
#                 key0 = 'EMPTY'
#             else:
#                 key0 = key[0].strip()
#     #         print(key0)
#             keys_final.append(key0)

#             # Get values
#             value0 = 'EMPTY'
#             try:
#                 value_raw, value = get_value(current_item_updated, next_item_updated, dx)
#                 if len(value) == 0:
#                     value = 'EMPTY'
#                 else:
#                     value0 = value[0].strip()
#             except:
#                 value0 = 'EMPTY'

#             values.append(value0)

#     # Combine into dict 
#     list_of_k_v = dict(zip(keys_final, values))
# #     list_of_k_v = pd.DataFrame(list(zip(keys_final, values)), columns =['KEYS', 'VALUES'])
    
#     return list_of_k_v
        


In [ ]:
# def getList(d):
#     return d.keys()

In [ ]:
dx = df_use['PATH_DX_SPEC_DESC'].iloc[0]
print(dx)
dx

In [ ]:
subs = ['[\\r\\n]{0,3}[- ]*[ \w\(\)\-\+\;\,\'\.]+[:]+']
matches = dict(re.findall(r"({0})\s*(.*?)(?=\s*(?:{0}|$))".format("|".join(subs)), dx))
# matches = re.findall(r"([\\r\\n]{0,3}[- ]*[ \w\(\)\-\+\;\,\'\.]+[:]{1})\s*(.*?)(?=\s*(?:[\\r\\n]{0,3}[- ]*[ \w\(\)\-\+\;\,\'\.]+[:]+|$))", dx)

matches

In [ ]:
def strip_results(dictionary):
    rmv_list = [':', '-']
    keys = [word.translate({ord(x): ' ' for x in rmv_list}).strip() for word, initial in dictionary.items()]
    values = [initial.strip() for initial, initial in dictionary.items()]
    dict_out = dict(zip(keys, values))
    
    return dict_out

In [ ]:
def create_key_value_pairs_2(dx):
    subs = ['[\\r\\n]{0,3}[- ]*[ \w\(\)\-\+\;\,\'\.]+[:]+']
    regex = re.compile(r"({0})\s*(.*?)(?=\s*(?:{0}|$))".format("|".join(subs)))
    matches = dict(regex.findall(dx))
    dict_out = strip_results(dictionary=matches)
    
    return dict_out

In [ ]:
list_of_k_v = create_key_value_pairs_2(dx=dx)

In [ ]:
list_of_k_v

In [ ]:
df_use.shape

In [ ]:
df_test = df_use[df_use['PATH_DX_SPEC_DESC'].notnull()]
kv_dict = df_test['PATH_DX_SPEC_DESC'].apply(lambda x: create_key_value_pairs_2(dx=x))
df_test = df_test.assign(PATH_DX_SPEC_DESC_DICT=kv_dict)

In [ ]:
df_test.iloc[410]['PATH_DX_SPEC_DESC_DICT']

In [ ]:
# def gather_key_headings(df):
series_dict = df_test['PATH_DX_SPEC_DESC_DICT'].apply(lambda x: getList(d=x))
df_names = pd.DataFrame(series_dict.to_list())
for i,col in enumerate(df_names.columns):
    if i == 0:
        df_long = df_names[col]
    else:
        df_long = pd.concat([df_long, df_names[col]], axis=0)

df_long1 = df_long.dropna()

df_column_names = df_long1.value_counts().reset_index()
df_column_names = df_column_names.rename(columns={'index': 'ATTRIBUTE', 0:'COUNTS'})
# pct = df_column_names[self._col_counts]/df_column_names[self._col_counts].sum()
# df_column_names = df_column_names.assign(pct=pct)

#     return df_column_names

df_column_names.head(10)

In [ ]:
df_test[df_test['PATH_DX_SPEC_DESC_DICT'].astype(str).str.contains('|'.join(['STAG', 'AJCC']))]
# df_test[df_test['PATH_DX_SPEC_DESC_DICT'].astype(str).str.contains('Histologic Grade')]

In [ ]:
df_column_names[df_column_names['ATTRIBUTE'].str.upper().str.contains('|'.join(['STAG', 'AJCC']))].head(30)

In [ ]:
df_column_names[df_column_names['ATTRIBUTE'].str.upper().str.contains('|'.join(['GRADE']))]

In [ ]:
ind = 99
print(df_test['PATH_DX_SPEC_DESC'].loc[ind])
print(df_test['PATH_DX_SPEC_DESC_DICT'].loc[ind])

In [ ]:
new_df = pd.DataFrame(df_test['PATH_DX_SPEC_DESC_DICT'].tolist(), index=df_test.index)
new_df = pd.concat([df_test[['ACCESSION_NUMBER', 'PATH_DX_SPEC_NUM']], new_df], axis=1)
new_df;

In [ ]:
new_df.head()

In [ ]:
new_df.to_csv('path_synoptic_dict_all.tsv', index=False, sep='\t')

In [ ]:
cols_index = ['ACCESSION_NUMBER', 'PATH_DX_SPEC_NUM']
logic_cols_use = new_df.columns.str.upper().str.contains('|'.join(['STAG', 'AJCC']))
cols_use = new_df.columns[logic_cols_use].to_list()
t = new_df[cols_index + cols_use].dropna(how='all', axis=0)
t_melt = t.melt(id_vars=cols_index, value_vars=cols_use, value_name='RESULT_STAGE', var_name='COL_NAME_STAGE')
t_melt = t_melt[t_melt['RESULT_STAGE'].notnull()]
print(t_melt.shape)
print(t_melt['COL_NAME_STAGE'].value_counts().head(10))
t_melt.head()

In [ ]:
logic_cols_use
cols_use = new_df.columns[logic_cols_use]
cols_use.to_list()

In [ ]:
t_missing = t_melt[t_melt['RESULT_STAGE'] == 'EMPTY'].merge(right=df_use, how='left', on=['ACCESSION_NUMBER' ,'PATH_DX_SPEC_NUM'])
t_notmissing = t_melt[t_melt['RESULT_STAGE'] != 'EMPTY'].merge(right=df_use, how='left', on=['ACCESSION_NUMBER' ,'PATH_DX_SPEC_NUM'])

In [ ]:
t_missing['PATH_DX_SPEC_DESC'].iloc[1]
# print(t_notmissing['PATH_DX_SPEC_DESC'].iloc[1])
t_missing['PATH_DX_SPEC_DESC'].iloc[1]

In [ ]:
df_impact_m = df_impact.merge(right=t_melt, how='left', left_on=['SOURCE_ACCESSION_NUMBER_0', 'SOURCE_SPEC_NUM_0'], right_on=['ACCESSION_NUMBER', 'PATH_DX_SPEC_NUM'])

In [ ]:
(df_impact_m['RESULT_STAGE'].notnull() & ).sum()/df_impact_m.shape[0]

In [ ]:
df_impact_m[df_impact_m['RESULT_STAGE'].notnull()]

In [ ]:
# new_df = pd.concat(pd.DataFrame({'name':k, 'value':v}) for k, v in x.items())
# df['index'] = df.groupby('name').cumcount().add(1)
# df

In [ ]:
df_surgical = df[df['ACCESSION_NUMBER'].str.contains('S')].reset_index(drop=True)
df_surgical.head()

In [ ]:
def get_df_kv(df):
    for i in range(df.shape[0]):
        current_kv_df = create_key_value_pairs(df['PATH_DX_SPEC_DESC'].iloc[i])
        current_kv_df = current_kv_df.assign(ACCESSION_NUMBER=df['ACCESSION_NUMBER'].iloc[i])
        current_kv_df = current_kv_df.assign(PATH_DX_SPEC_NUM=df['PATH_DX_SPEC_NUM'].iloc[i])

        if i == 0:
            df_kv_final = current_kv_df.copy()
        else:
            df_kv_final = pd.concat([df_kv_final, current_kv_df], axis=0)
    #     print(df_test['PATH_DX_SPEC_DESC'][i])
    #     print('----')
    return df_kv_final



In [ ]:
df_kv_final = get_df_kv(df=df_test)
df_kv_final.to_csv('pathology_spec_desc_key_value.tsv', index=False, sep='\t')

In [ ]:
df_kv_final.head(50)